In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

In [2]:
data_train=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train').read_all()
random.shuffle(data_train)
#data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
#data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [3]:
df_train=pd.DataFrame(data_train)
#df_val=pd.DataFrame(data_val)
#df_test=pd.DataFrame(data_test)

In [4]:
df_train.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-tmp/coco2017/train2017/000000214...,[Question:At least how many people can ride an...,"[tensor(1), tensor(0), tensor(2), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
1,[/root/autodl-tmp/coco2017/train2017/000000183...,[Question:What kind of walls are in this house...,"[tensor(2), tensor(2), tensor(1), tensor(2), t...",{'embedding': [[[-0.01116943 0.00212097 -0.00...,None
2,[/root/autodl-tmp/coco2017/train2017/000000569...,[Question:The rightmost set of rails leads to ...,"[tensor(3), tensor(1), tensor(2), tensor(3), t...",{'embedding': [[[ 0.01013184 0.00717163 -0.00...,None
3,[/root/autodl-tmp/coco2017/train2017/000000401...,[Question:What kind of buildings are the ones ...,"[tensor(0), tensor(2), tensor(1), tensor(1), t...",{'embedding': [[[ 0.00915527 0.01464844 0.03...,None
4,[/root/autodl-tmp/coco2017/train2017/000000558...,[Question:What are the three zebras in the wat...,"[tensor(2), tensor(2), tensor(3), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None


In [5]:
i=0
df_train.iloc[i]['most_likely']['response']

['B', 'A', 'C', 'A', 'B', 'A.blueberry', 'A', 'A']

In [6]:
df_train.iloc[i]['img_path']

['/root/autodl-tmp/coco2017/train2017/000000214699.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000578962.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000549042.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000329963.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000081476.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000089638.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000418854.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000044079.jpg']

In [7]:
df_train.iloc[i]['question']

['Question:At least how many people can ride an elephant at once?\nAnswer the Question with following choices.\nChoices: A.ten B.three C.eight D.five\n',
 'Question:Which country is this street station a part of?\nAnswer the Question with following choices.\nChoices: A.thailand B.greece C.georgia D.romania\n',
 'Question:What is the elephant wearing?\nAnswer the Question with following choices.\nChoices: A.basket B.hat C.blue ribbon D.crown\n',
 'Question:What are the two horns on this animal called?\nAnswer the Question with following choices.\nChoices: A.ossicones B.antlers C.pedicles D.scurs\n',
 'Question:What is by the water?\nAnswer the Question with following choices.\nChoices: A.cats B.polar bears C.foxes D.wolves\n',
 'Question:What would a person eat if they only eat food that is the same color as the topmost item in the red bowl?\nAnswer the Question with following choices.\nChoices: A.blueberry B.cherry C.salad D.oranges\n',
 'Question:Why are the walls tiled?\nAnswer the Q

In [8]:
df_train.iloc[0]['answer']

tensor([1, 0, 2, 0, 1, 2, 2, 0])

In [9]:
embeddings=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(17056, 33, 4096)

In [10]:
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(17056,)

In [11]:
gt_label[0]

np.int64(1)

In [12]:
judge=load_judge('OKVQA')

In [13]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_correct=[]
for i,x in enumerate(rsps):
    is_correct.append(judge.check(x,gt_label[i]))

is_correct=np.array(is_correct)
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 17056 responses. 11081 answer correctly and 5975 answer wrongly


In [14]:
length = len(embeddings)
train_rate = 0.8
num_val=500

embedding_train = embeddings[:int(length*train_rate)-num_val]
embedding_val = embeddings[int(length*train_rate)-num_val:int(length*train_rate)]
embedding_test = embeddings[int(length*train_rate):]


is_correct_train = is_correct[:int(length*train_rate)-num_val]
is_correct_val = is_correct[int(length*train_rate)-num_val:int(length*train_rate)]
is_correct_test = is_correct[int(length*train_rate):]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(is_correct_train)}')
print(f'validset length:{len(is_correct_val)}')
print(f'testset length:{len(is_correct_test)}')
print(
    f'trainset postive: {sum(is_correct_train==1)} negative: {sum(is_correct_train==0)}')
print(
    f'valset postive: {sum(is_correct_val==1)} negative: {sum(is_correct_val==0)}')
print(
    f'testset postive: {sum(is_correct_test==1)} negative: {sum(is_correct_test==0)}')

trainset length:13144
validset length:500
testset length:3412
trainset postive: 8576 negative: 4568
valset postive: 346 negative: 154
testset postive: 2159 negative: 1253


In [15]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [16]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [17]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,is_correct_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5210757450641844


  9%|▉         | 1/11 [00:00<00:07,  1.37it/s]

found best n_components:2,layer:0 Score: 0.555222956234517


 18%|█▊        | 2/11 [00:01<00:07,  1.22it/s]

found best n_components:3,layer:0 Score: 0.5720760453419413


100%|██████████| 11/11 [00:12<00:00,  1.18s/it]


In [18]:
best_paras

{'n_components': 3, 'layer': 0, 'best_score': np.float64(0.5720760453419413)}

In [19]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [20]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(is_correct_val)

In [21]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(is_correct_train,y_train)} AUROC:{roc_auc_score(is_correct_train,scores)} AUC-PR:{aur_pr(scores,is_correct_train)}')

train set acc: 0.6463785757760194 AUROC:0.5196159337466019 AUC-PR:0.6881588871557972


In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
y_test=(scores>best_split)
print(f'test set acc: {accuracy_score(is_correct_test,y_test)} AUROC:{roc_auc_score(is_correct_test,scores)} AUC-PR:{aur_pr(scores,is_correct_test)}')

test set acc: 0.6099062133645955 AUROC:0.4978282783662886 AUC-PR:0.6561419856373895


Train Linear Prob to Get Better Results

In [23]:
X_train=embedding_train[:, best_layer,:]

X_test=embedding_test[:, best_layer,:]

In [24]:
best_acc, final_acc, (
            clf, best_state, best_preds, y_train, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
             y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [25]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
y_preds=scores>0.5
print(f'test set acc: {accuracy_score(is_correct_test,y_preds)} AUROC:{roc_auc_score(is_correct_test,scores)} AUC-PR:{aur_pr(scores,is_correct_test)} f1:{f1_score(is_correct_test,y_preds)}')

test set acc: 0.6327667057444314 AUROC:0.47663800486983166 AUC-PR:0.6443647411016443 f1:0.7750852629689463
